In [3]:
import gurobipy as gb
from gurobipy import GRB

# Create a new model
model = gb.Model("Sunnyshore Bay Financial Planning")

# Decision variables:
# b_ij = amount borrowed in month i to be paid in month j
# z_i = cash balance at the end of month i

# Define months
months = ["May", "June", "July", "August"]
month_nums = [1, 2, 3, 4]  # May=1, June=2, July=3, August=4

# Cash balance at the end of each month
z = model.addVars(month_nums, lb=0, vtype=GRB.CONTINUOUS, name="Cash_Balance")

# Amount borrowed in month i to be paid in month j
b = {}
for i in range(1, 4):  # Can borrow in May, June, July
    for j in range(i+1, 5):  # Payment months
        if j - i <= 3:  # Can only borrow for up to 3 months
            b[i, j] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"Borrow_{months[i-1]}_Pay_{months[j-1]}")

# Revenues and expenses for each month
revenues = [180000, 260000, 420000, 580000]
expenses = [300000, 400000, 350000, 200000]

# Interest rates (as multipliers)
interest_rates = {1: 1.0175, 2: 1.0225, 3: 1.0275}

# Objective: Minimize total repayment
objective = gb.quicksum(interest_rates[j-i] * b[i, j] for i, j in b)
model.setObjective(objective, GRB.MINIMIZE)

# Initial cash balance
initial_balance = 140000

# Cash balance constraints
# May (Month 1)
model.addConstr(z[1] == initial_balance + revenues[0] - expenses[0] + 
                gb.quicksum(b[1, j] for j in range(2, 5) if (1, j) in b), "Balance_May")

# June (Month 2)
model.addConstr(z[2] == z[1] + revenues[1] - expenses[1] + 
                gb.quicksum(b[2, j] for j in range(3, 5) if (2, j) in b) - 
                interest_rates[1] * b[1, 2], "Balance_June")

# July (Month 3)
model.addConstr(z[3] == z[2] + revenues[2] - expenses[2] + 
                gb.quicksum(b[3, j] for j in range(4, 5) if (3, j) in b) - 
                interest_rates[1] * b[2, 3] - 
                interest_rates[2] * b[1, 3], "Balance_July")

# August (Month 4)
model.addConstr(z[4] == z[3] + revenues[3] - expenses[3] - 
                interest_rates[1] * b[3, 4] - 
                interest_rates[2] * b[2, 4] - 
                interest_rates[3] * b[1, 4], "Balance_August")

# Minimum cash balance requirements
min_balance = [25000, 20000, 35000, 18000]
for i in range(1, 5):
    model.addConstr(z[i] >= min_balance[i-1], f"Min_Balance_{months[i-1]}")

# Maximum borrowing limits
borrow_limits = [250000, 150000, 350000]
for i in range(1, 4):
    model.addConstr(gb.quicksum(b[i, j] for j in range(i+1, 5) if (i, j) in b) <= borrow_limits[i-1], 
                   f"Max_Borrow_{months[i-1]}")

# Cash balance at end of July must be at least 65% of combined totals from May and June
model.addConstr(z[3] >= 0.65 * (z[1] + z[2]), "July_Balance_Ratio")

# Optimize the model
model.optimize()

# Print results
if model.status == GRB.OPTIMAL:
    print(f"Total repayment: ${model.objVal:.2f}")
    
    print("\nBorrowing Amounts:")
    for i, j in b:
        if b[i, j].x > 0:
            print(f"Borrow in {months[i-1]}, repay in {months[j-1]}: ${b[i, j].x:.2f} (Repayment: ${b[i, j].x * interest_rates[j-i]:.2f})")
    
    print("\nCash Balances:")
    for i in range(1, 5):
        print(f"End of {months[i-1]}: ${z[i].x:.2f}")
    
    # Total borrowing per month
    for i in range(1, 4):
        total_borrowed = sum(b[i, j].x for j in range(i+1, 5) if (i, j) in b)
        print(f"\nTotal borrowed in {months[i-1]}: ${total_borrowed:.2f}")
else:
    print("No optimal solution found.")

# Get constraint information for sensitivity analysis
june_balance_constraint = model.getConstrByName("Min_Balance_June")
print(f"\nSensitivity Analysis for June Minimum Balance Constraint:")
print(f"Shadow Price: {june_balance_constraint.pi}")
print(f"RHS Value: {june_balance_constraint.RHS}")
print(f"Allowable Increase: {june_balance_constraint.SARHSUp}")
print(f"Allowable Decrease: {june_balance_constraint.SARHSLow}")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 10 columns and 32 nonzeros
Model fingerprint: 0x727a263d
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 5 rows and 1 columns
Presolve time: 0.00s
Presolved: 7 rows, 9 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.812500e+04   0.000000e+00      0s
       6    1.4290473e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.429047297e+05
Total repayment: $142904.73

Borrowing Amounts:
Borrow in May, repay in August: $5000.00 (Repayment: $5137.50)
Borrow in June, repay in July: $54054.05 (Repayment: $55000.00)
Borrow 